In [1]:
from ortools.linear_solver import pywraplp
import random
solver = pywraplp.Solver.CreateSolver('SCIP')

n = 2
m = 7
q = 13
d = (q - 1) // 2

L = {}
for j in range(m):
	for i in range(n):
		L[j,i] = random.randint(-d, d+1)


# Vetor de alocação
e = [ solver.IntVar(-1, 1, f"e[{j}]") for j in range(m) ]

# restrições
for i in range(n):
	K = solver.IntVar(0, q, f'K[{i}]')
	solver.Add(sum(e[j] * L[j, i] for j in range(m)) == q * K)


# implementação do boolean cast
# https://cs.stackexchange.com/a/51089
u = [ solver.BoolVar(f'u[{j}]') for j in range(m) ]

for j in range(m):
	delta = solver.BoolVar(f'delta[{j}]')
	solver.Add(u[j] - 2 * delta <= e[j])
	solver.Add(e[j] <= -u[j] + 2 * (1 - delta))

solver.Add(sum(u[j] for j in range(m)) >= 1)


# minimizar o número de componentes nulas
# i.e. maximizar o número de componentes não nulas
solver.Maximize(sum(u[j] for j in range(m)))


status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
	print("e: ", end="")
	for j in range(m):
		print(f'{int(e[j].solution_value()):2}', end=' ')
	print()

	print("u: ", end="")
	for j in range(m):
		print(f'{int(u[j].solution_value()):2}', end=' ')
	print()


else:
	print("unsat")



e: -1  0  1  1  1  1  1 
u:  1  0  1  1  1  1  1 
